In [ ]:
1+1

In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

import pid
from pyanalib import panda_helpers
import os
import multiprocessing


In [ ]:
from pyanalib.dataset import Dataset

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
mpl.rc('font', size=14)

dosave = False
savedir = "./plots_10_4_23/"

import warnings

warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [ ]:
datadir = "/icarus/data/users/gputnam/calib-data/outputs/"

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/"

onbeamfile_Run1 = filedir + "data-reprod-D/NuMIMajorityRun1OnBeam_evt.df"
onbeamfile_Run2 = filedir + "data-reprod-D/NuMIMajorityRun2OnBeam_evt.df"

offbeamfile_Run1 = filedir + "data-reprod-D/NuMIMajorityRun1OffBeam_evt.df"
offbeamfile_Run2 = filedir + "data-reprod-D/NuMIMajorityRun2OffBeam_evt.df"

mcfile = "/icarus/data/users/gputnam/DMCP2023G/mc/mcnuphase2_evt_reprodD.df"

In [ ]:
mc = data.mc_dataset(mcfile, "evt")
mc.df

In [ ]:
mc.df["Run1"] = True
mc.df["Run2"] = True

In [ ]:
onbeamR1 = data.onbeam_dataset(onbeamfile_Run1, "evt")
onbeamR2 = data.onbeam_dataset(onbeamfile_Run2, "evt")

In [ ]:
onbeamR1.df["Run1"] = True
onbeamR1.df["Run2"] = False

onbeamR2.df["Run1"] = False
onbeamR2.df["Run2"] = True

onbeamR1.df = onbeamR1.df.iloc[::2]
onbeamR1.POT = onbeamR1.POT / 2
onbeamR1.livetime = onbeamR1.livetime / 2

onbeamR2.df = onbeamR2.df.iloc[::2]
onbeamR2.POT = onbeamR2.POT / 2
onbeamR2.livetime = onbeamR2.livetime / 2

onbeam = onbeamR1.concat(onbeamR2)

In [ ]:
offbeamR1 = data.offbeam_dataset(offbeamfile_Run1, "evt")
offbeamR2 = data.offbeam_dataset(offbeamfile_Run2, "evt")

In [ ]:
offbeamR1.df["Run1"] = True
offbeamR1.df["Run2"] = False

offbeamR2.df["Run1"] = False
offbeamR2.df["Run2"] = True

offbeam = offbeamR1.concat(offbeamR2)

In [ ]:
POTSTR = "$%.2f \\times 10^{20}$ POT" % (onbeam.POT/1e20)
POTSTR

In [ ]:
trunkIDcategories = [
    ((var.slc.truth.pdg == 14) | (var.slc.truth.pdg == -14)) & (var.slc.truth.iscc==1) & ((var.slc.truth.genie_mode == 0) | (var.slc.truth.genie_mode == 10)), # CC QE+MEC
    ((var.slc.truth.pdg == 14) | (var.slc.truth.pdg == -14)) & (var.slc.truth.iscc==1) & (var.slc.truth.genie_mode == 1), # CC RES
    ((var.slc.truth.pdg == 14) | (var.slc.truth.pdg == -14)) & (var.slc.truth.iscc==1) & (var.slc.truth.genie_mode == 2), # CC DIS
    ((var.slc.truth.pdg == 14) | (var.slc.truth.pdg == -14)) & (var.slc.truth.iscc==1) & (var.slc.truth.genie_mode == 3), # CC COH
    ((var.slc.truth.iscc == 0) & (var.slc.tmatch.idx >= 0)), # NC
]
nucategories.append(~nucategories[0] & (var.slc.tmatch.idx >= 0))
for c in nucategories[1:-1]:
    nucategories[-1] = nucategories[-1] & ~c
    
nucategories.append(var.slc.tmatch.idx < 0)
    
nucategories[0].name = "$\\nu_\\mu$ CC QE+MEC"
nucategories[1].name = "$\\nu_\\mu$ CC RES"
nucategories[2].name = "$\\nu_\\mu$ CC DIS"
nucategories[3].name = "$\\nu_\\mu$ CC COH"
nucategories[4].name = "$\\nu$ NC"
nucategories[5].name = "$\\nu$ Other"
nucategories[6].name = "Cosmic"

In [ ]:
trunkIDcategories = [
    (var.DF.trunk.trk.truth.p.pdg == 13) | (var.DF.trunk.trk.truth.p.pdg == -13),
    (var.DF.trunk.trk.truth.p.pdg == 211) | (var.DF.trunk.trk.truth.p.pdg == -211),
    (var.DF.trunk.trk.truth.p.pdg == 2212),
    (var.DF.trunk.trk.truth.p.pdg == 22),
]
trunkIDcategories.append(~trunkIDcategories[0])
for c in trunkIDcategories[1:-1]:
    trunkIDcategories[-1] = trunkIDcategories[-1] & ~c
    
trunkIDcategories[0].name = "$\\mu$"
trunkIDcategories[1].name = "$\\pi$"
trunkIDcategories[2].name = "$p$"
trunkIDcategories[3].name = "$\\gamma$"
trunkIDcategories[4].name = "Other"

In [ ]:
branchIDcategories = [
    (var.DF.branch.trk.truth.p.pdg == 13) | (var.DF.branch.trk.truth.p.pdg == -13),
    (var.DF.branch.trk.truth.p.pdg == 211) | (var.DF.branch.trk.truth.p.pdg == -211),
    (var.DF.branch.trk.truth.p.pdg == 2212),
    (var.DF.branch.trk.truth.p.pdg == 22),
]
branchIDcategories.append(~branchIDcategories[0])
for c in branchIDcategories[1:-1]:
    branchIDcategories[-1] = branchIDcategories[-1] & ~c
    
branchIDcategories[0].name = "$\\mu$"
branchIDcategories[1].name = "$\\pi$"
branchIDcategories[2].name = "$p$"
branchIDcategories[3].name = "$\\gamma$"
branchIDcategories[4].name = "Other"

In [ ]:
vtxIDcategories = [
    (trunkIDcategoires[0] | trunkIDcategoires[1]) & (branchIDcategoires[0] | branchIDcategoires[1]),
    ((trunkIDcategoires[0] | trunkIDcategoires[1]) & branchIDcategoires[2]) |\
    (trunkIDcategoires[2] & (branchIDcategoires[0] | branchIDcategoires[1])),
]

vtxIDcategories.append(~vtxIDcategories[0])
for c in branchIDcategoires[1:-1]:
    vtxIDcategories[-1] = vtxIDcategories[-1] & ~c

vtxIDcategories[0].name = "$\\mu\\pi$, $\\pi\\pi$"
vtxIDcategories[1].name = "$\\mu p$, $\\pi p$"
vtxIDcategories[2].name = "Other"

In [ ]:
def chi2(mc, onbeam, offbeam, scov):
    Ndata = onbeam.N - offbeam.N
    Nmc = mc.N
    stat_err = mc.N + offbeam.Nerr**2
    
    cov = (scov + np.diag(stat_err))[np.ix_(np.where(mc.N != 0)[0], np.where(mc.N != 0)[0])]
    delta = (Nmc-Ndata)[Nmc != 0]
    
    return np.matmul(np.matmul(delta, np.linalg.inv(cov)), delta), int(np.sum(mc.N != 0))

In [ ]:
def mcdata_plot(v, bins, c, categories=None, xlabel=None, ylabel=None):
    hnom, hsyst = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT, syst="all", 
                            categories=categories)
    hcov = hist.makecovariance(sum(hnom), hsyst)
    hon, _ = hist.makehist(v, onbeam, cut=c, bins=bins, cvweight=False, syst=None)
    hoff, _ = hist.makehist(v, offbeam, cut=c, bins=bins, cvweight=False, livetime=onbeam.livetime, syst=None)

    fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
    fig.subplots_adjust(hspace=0.03)

    hmc, hfill = hist.plotmc(p0, hnom, hcov)
    _, _, fh = hmc
    hd = hist.plotdata(p0, hon-hoff)
    p0.set_ylim([p0.get_ylim()[0], p0.get_ylim()[1]*1.25])

    legend1 = p0.legend(handles=[hd], labels=["Data"], loc="upper left", frameon=False)
    p0.legend(handles=fh, labels=[c.name for c in categories], 
                        loc="upper center", ncol=3, bbox_to_anchor=(0.5, 1.35))
    p0.add_artist(legend1)

    tchi2, ndof = chi2(sum(hnom), hon, hoff, hcov)

    p0.text(0.6, 0.9, "$\\chi^2/n$ = %.1f / %i" % (tchi2, ndof), fontsize=14,
            transform=p0.transAxes)

    hist.plotratio(plt, hon-hoff, sum(hnom), hcov)
    p1.axhline([1], color="r", linestyle=":")
    p1.set_ylim([0.5, 1.5])

    if xlabel:
        p1.set_xlabel(xlabel)
    if ylabel:
        p0.set_ylabel("%s / %s" % (ylabel, POTSTR))
        
    return fig, (p0, p1)

In [ ]:
def recop(trk):
    p = trk.rangeP.p_muon
    p[~TrkInFV(trk.end)] = trk.mcsP.fwdP_muon
    return p

beam2det = np.array([ [0.921035925, 0.022715103, 0.388814672], [0, 0.998297825, -0.058321970], [-0.389477631, 0.053716629, 0.919468161]])

beamorigin = np.array([4.503730e2, 80.153901e2, 795.112945e2])

MUON_MASS = 0.1057 
PION_MASS = 0.13457

BEAMDIR = beam2det.dot(beamorigin) / np.linalg.norm(beam2det.dot(beamorigin)) 
BEAMDIR

In [ ]:
offbeam.df["trunk_mcs_range"] = np.abs(1 - offbeam.df.trunk.trk.mcsP.fwdP_muon / offbeam.df.trunk.trk.rangeP.p_muon)
onbeam.df["trunk_mcs_range"] = np.abs(1 - onbeam.df.trunk.trk.mcsP.fwdP_muon / onbeam.df.trunk.trk.rangeP.p_muon)
mc.df["trunk_mcs_range"] = np.abs(1 - mc.df.trunk.trk.mcsP.fwdP_muon / mc.df.trunk.trk.rangeP.p_muon)

offbeam.df["branch_mcs_range"] = np.abs(1 - offbeam.df.branch.trk.mcsP.fwdP_muon / offbeam.df.branch.trk.rangeP.p_muon)
onbeam.df["branch_mcs_range"] = np.abs(1 - onbeam.df.branch.trk.mcsP.fwdP_muon / onbeam.df.branch.trk.rangeP.p_muon)
mc.df["branch_mcs_range"] = np.abs(1 - mc.df.branch.trk.mcsP.fwdP_muon / mc.df.branch.trk.rangeP.p_muon)

In [ ]:
offbeam.df["openangle"] = np.arccos(dotdf(offbeam.df.trunk.trk.dir, offbeam.df.branch.trk.dir))*180/np.pi
onbeam.df["openangle"] = np.arccos(dotdf(onbeam.df.trunk.trk.dir, onbeam.df.branch.trk.dir))*180/np.pi
mc.df["openangle"] = np.arccos(dotdf(mc.df.trunk.trk.dir, mc.df.branch.trk.dir))*180/np.pi

In [ ]:
off_scalar_mom = offbeam.df.trunk.trk.dir.multiply(recop(offbeam.df.trunk.trk), axis=0) + \
           offbeam.df.branch.trk.dir.multiply(recop(offbeam.df.branch.trk), axis=0)
off_scalar_dir = off_scalar_mom.divide(magdf(off_scalar_mom), axis=0)
offbeam.df["beamangle"] = (180/np.pi)*np.arccos(off_scalar_dir.x*BEAMDIR[0] + off_scalar_dir.y*BEAMDIR[1] + off_scalar_dir.z*BEAMDIR[2])


on_scalar_mom = onbeam.df.trunk.trk.dir.multiply(recop(onbeam.df.trunk.trk), axis=0) + \
           onbeam.df.branch.trk.dir.multiply(recop(onbeam.df.branch.trk), axis=0)
on_scalar_dir = on_scalar_mom.divide(magdf(on_scalar_mom), axis=0)
onbeam.df["beamangle"] = (180/np.pi)*np.arccos(on_scalar_dir.x*BEAMDIR[0] + on_scalar_dir.y*BEAMDIR[1] + on_scalar_dir.z*BEAMDIR[2])


mc_scalar_mom = mc.df.trunk.trk.dir.multiply(recop(mc.df.trunk.trk), axis=0) + \
           mc.df.branch.trk.dir.multiply(recop(mc.df.branch.trk), axis=0)
mc_scalar_dir = mc_scalar_mom.divide(magdf(mc_scalar_mom), axis=0)
mc.df["beamangle"] = (180/np.pi)*np.arccos(mc_scalar_dir.x*BEAMDIR[0] + mc_scalar_dir.y*BEAMDIR[1] + mc_scalar_dir.z*BEAMDIR[2])


In [ ]:
off_Etrunk = np.sqrt(MUON_MASS**2 + recop(offbeam.df.trunk.trk)**2)
off_Ebranch = np.sqrt(MUON_MASS**2 + recop(offbeam.df.branch.trk)**2)
offbeam.df["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + off_Etrunk*off_Ebranch - recop(offbeam.df.trunk.trk)*recop(offbeam.df.branch.trk)*dotdf(offbeam.df.branch.trk.dir, offbeam.df.trunk.trk.dir))

on_Etrunk = np.sqrt(MUON_MASS**2 + recop(onbeam.df.trunk.trk)**2)
on_Ebranch = np.sqrt(MUON_MASS**2 + recop(onbeam.df.branch.trk)**2)
onbeam.df["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + on_Etrunk*on_Ebranch - recop(onbeam.df.trunk.trk)*recop(onbeam.df.branch.trk)*dotdf(onbeam.df.branch.trk.dir, onbeam.df.trunk.trk.dir))

mc_Etrunk = np.sqrt(MUON_MASS**2 + recop(mc.df.trunk.trk)**2)
mc_Ebranch = np.sqrt(MUON_MASS**2 + recop(mc.df.branch.trk)**2)
mc.df["s_mass"] = np.sqrt(2)*np.sqrt(MUON_MASS**2 + mc_Etrunk*mc_Ebranch - recop(mc.df.trunk.trk)*recop(mc.df.branch.trk)*dotdf(mc.df.branch.trk.dir, mc.df.trunk.trk.dir))

In [ ]:
othr_obj_cut = ~((var.DF.max_shw_len > 15) | (var.DF.max_othr_trk_len > 20) |\
                 (var.DF.min_othr_chi2_proton < 80) | (var.DF.max_othr_chi2_muon > 40))

trunk_muid = (var.DF.trunk.trk.len > 25) & (var.DF.trunk.trk.chi2pid.I2.chi2_muon < 15) &\
    (var.DF.trunk.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.trunk_mcs_range < 0.5)
branch_muid = (var.DF.branch.trk.len > 25) & (var.DF.branch.trk.chi2pid.I2.chi2_muon < 15) &\
    (var.DF.branch.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.branch_mcs_range < 0.5)

hasstub = (var.DF.stub.l0_5cm.dqdx > 5e5) | (var.DF.stub.l1cm.dqdx > 3e5) |\
    (var.DF.stub.l2cm.dqdx > 2.5e5) | (var.DF.stub.l3cm.dqdx > 2e5)

stubid = ~hasstub


In [ ]:
cuts = [
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25),
    
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     (var.crlongtrkdiry@var.slc > -0.4),
    
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     (var.crlongtrkdiry@var.slc > -0.4) &\
#     (var.DF.slc.producer == 0),
    
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     (var.crlongtrkdiry@var.slc > -0.4) &\
#     (var.DF.slc.producer == 1),
    
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     trunk_muid & branch_muid,

#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     trunk_muid & branch_muid,
    
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     (var.DF.openangle < 100) &\
#     trunk_muid & branch_muid,
    
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     (var.DF.openangle < 100) &\
#     trunk_muid & branch_muid &\
#     (var.DF.slc.producer == 0),
    
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     (var.DF.openangle < 100) &\
#     trunk_muid & branch_muid &\
#     (var.DF.slc.producer == 1),
    
    cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.beamangle > 25) &\
    (var.DF.openangle < 100) &\
    trunk_muid & branch_muid & othr_obj_cut,
    
#     cut.fiducial@var.slc &\
#     cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
#     (var.DF.beamangle > 25) &\
#     (var.DF.openangle < 100) &\
#     trunk_muid & branch_muid & othr_obj_cut & stubid,
]

In [ ]:
# N0,B0,_ = plt.hist(mc.df.slc.truth.E[cuts[0](mc.df)], bins=np.linspace(0, 10, 41), histtype="step", density=True)
# _ = plt.hist(mc.df.slc.truth.E[cuts[1](mc.df)], bins=np.linspace(0, 10, 41), histtype="step", density=True)

In [ ]:
mc.POT

In [ ]:
# hnom, hsyst = hist.makehist(v, mc, cut=c, bins=bins, POT=onbeam.POT, syst=True, 
#                         categories=nucategories)

In [ ]:
# centers = (bins[:-1] + bins[1:]) / 2

In [ ]:
# for i in range(10):
#     _ = plt.hist(centers, bins=bins, weights=hsyst[i].N, histtype="step", linewidth=2)

In [ ]:
v = var.DF.openangle
c =  cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.beamangle > 25) &\
    trunk_muid & branch_muid

bins = np.linspace(0, 180, 21)

mcdata_plot(v, bins, c, nucategories, "Opening Angle", "Slices")

In [ ]:
cut_text = [    
#     "Pre-Selection",
#     "CRLongTrkDirY > -0.4",
#     "CRLongTrkDirYCut\nEast Cryostat",
#     "CRLongTrkDirYCut\nWest Cryostat",
#     "Tight Muon ID",
#     "Tight Muon ID +\nOpen Angle < 100$^\\circ$",
#     "Tight Muon ID +\nOpen Angle < 100$^\\circ$\nEast Cryostat",
#     "Tight Muon ID +\nOpen Angle < 100$^\\circ$\nWest Cryostat",
    "Loose Muon ID +\nOther Obj Cut",
#     "Tight Muon ID +\nOther Obj Cut +\nStub ID",
]

In [ ]:
cut_dir = [    
#     "PreSelection",
#     "CRLongTrkDirYCut",
#     "CRLongTrkDirYCut_CryoE",
#     "CRLongTrkDirYCut_CryoW",
#     "TightMuonID",
#     "TightMuonID_OpenAngle",
#     "TightMuonID_OpenAngle_CryoE",
#     "TightMuonID_OpenAngle_CryoW",
    "LooseMuonID_OthrObj",
#     "TightMuonID_OthrObj_Stub"
]

In [ ]:
plotvars = [
    var.crlongtrkdiry@var.slc,
    var.DF.max_shw_len,
    var.DF.max_othr_trk_len,
    var.DF.max_othr_chi2_muon,
    var.DF.min_othr_chi2_proton,
    var.DF.trunk.trk.len,
    var.DF.branch.trk.len,
    var.DF.trunk.trackScore,
    var.DF.branch.trackScore,
    var.DF.trunk.trk.chi2pid.I2.chi2_muon,
    var.DF.branch.trk.chi2pid.I2.chi2_muon,
    var.DF.trunk.trk.chi2pid.I2.chi2_proton,
    var.DF.branch.trk.chi2pid.I2.chi2_proton,
    var.DF.trunk_mcs_range,
    var.DF.branch_mcs_range,
    var.DF.stub.l0_5cm.dqdx,
    var.DF.stub.l1cm.dqdx,
    var.DF.stub.l2cm.dqdx,
    var.DF.stub.l3cm.dqdx,
    var.DF.openangle,
    var.DF.beamangle,
    var.DF.s_mass
]

In [ ]:
plotnames = [
    "CRLongTrkDirY",
    "Maximum Shower Length [cm]",
    "Maximum Other Track Length [cm]",
    "Maximum Other $\\chi^2_\\mu$",
    "Minimum Other $\\chi^2_p$",
    "Trunk Track Length [cm]",
    "Branch Track Length [cm]",
    "Trunk Track Score",
    "Branch Track Score",
    "Trunk $\\chi^2_\\mu$",
    "Branch $\\chi^2_\\mu$",
    "Trunk $\\chi^2_p$",
    "Branch $\\chi^2_p$",
    "Trunk (1 - MCS P / Range P)",
    "Branch (1 - MCS P / Range P)",
    "Stub dQ/dx (len < 0.5 cm) [ke$^-$/cm]",
    "Stub dQ/dx (0.5 < len < 1 cm) [ke$^-$/cm]",
    "Stub dQ/dx (1 < len < 2 cm) [ke$^-$/cm]",
    "Stub dQ/dx (2 < len < 3 cm) [ke$^-$/cm]",
    "Trunk - Branch Opening Angle [$^\\circ$]",
    "Summed Angle to Beam [$^\\circ$]",
    "$M_{\\mu\\mu}$ [GeV/c$^2$]"
]

In [ ]:
plotsave = [
    "crlongtrkdiry",
    "max_shw_len",
    "max_othr_trk_len",
    "max_othr_chi2u",
    "min_othr_chi2p",
    "trunk_trk_len",
    "branch_trk_len",
    "trunk_trk_score",
    "branch_trk_score",
    "trunk_chi2u",
    "branch_chi2u",
    "trunk_chi2p",
    "branch_chi2p",
    "trunk_mcsvrange",
    "branch_mcsvrange",
    "stub_dqdx_0_5cm",
    "stub_dqdx_1cm",
    "stub_dqdx_2cm",
    "stub_dqdx_3cm",
    "openangle",
    "beamangle",
    "s_mass",
]

In [ ]:
plotbins = [
    np.linspace(-1, 1, 21),
    np.linspace(0, 20, 21),
    np.linspace(0, 60, 21),
    np.linspace(0, 60, 21),
    np.linspace(0, 60, 21),
    np.linspace(0, 200, 21) + 10,
    np.linspace(0, 200, 21) + 10,
    np.linspace(0, 1, 21),
    np.linspace(0, 1, 21),
    np.linspace(0, 30, 16),
    np.linspace(0, 30, 16),
    np.linspace(80, 400, 21),
    np.linspace(80, 400, 21),
    np.linspace(0, 1, 21),
    np.linspace(0, 1, 21),
    np.linspace(0, 15e5, 16),
    np.linspace(0, 15e5, 16),
    np.linspace(0, 15e5, 16),
    np.linspace(0, 15e5, 16),
    np.linspace(0, 180, 21),
    np.linspace(0, 140, 21) + 25,
    np.linspace(0, 0.5, 21) + 2*MUON_MASS
]

In [ ]:
# ifig = 0
# p = multiprocessing.Pool(10)

# def makeplt(inp):
#     v, name, savename, bins = inp
# #     plt.figure(0)

#     fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices")
#     p0.text(0.85, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")

#     if dosave: 
#         plt.savefig(os.path.join(savedir, cdir, ("%s.pdf" % savename)), bbox_inches="tight")
        
#     plt.close()
        
#     return n

# for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
#     print(ctxt)
#     if dosave:
#         os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
        
#     for n in p.imap_unordered(makeplt, zip(plotvars, plotnames, plotsave, plotbins)):
#         print(n)
        
#     break

In [ ]:
len(list(zip(cuts, cut_text, cut_dir)))

In [ ]:
#  chi2pid
whichvars = [
    5, 7, 9, 11, 13
]

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, trunkIDcategories, name, "Slices")
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("trunkcat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("trunkcat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
# Branch chi2pid
whichvars = [
    6, 8, 10, 12, 14
]

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, branchIDcategories, name, "Slices")
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("branchcat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("branchcat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
# stubs
whichvars = [
    15, 16, 17, 18
]

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, vtxIDcategories, name, "Slices")
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        scale_x = 1e3
        ticks_x = mpl.ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_x))
        p0.xaxis.set_major_formatter(ticks_x)
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("vtxcat_%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("vtxcat_%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1        
#     if dosave: 
#         plt.close("all")

In [ ]:
whichvars = list(range(len(plotvars))) # all

In [ ]:
ifig = 0

for c, ctxt, cdir in zip(cuts, cut_text, cut_dir):
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for ivar, (v, name, savename, bins) in enumerate(zip(plotvars, plotnames, plotsave, plotbins)):
        if ivar not in whichvars: continue
            
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices")
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("%s.pdf" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("%s.svg" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
othr_obj_cut = ~((var.DF.max_shw_len_primary > 15) | (var.DF.max_othr_trk_len > 20) |\
                 (var.DF.min_othr_chi2_proton_all < 80))

trunk_muid = (var.DF.trunk.trk.len > 25) & (var.DF.trunk.trk.chi2pid.I2.chi2_muon < 10) &\
    (var.DF.trunk.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.trunk_mcs_range < 0.5)
branch_muid = (var.DF.branch.trk.len > 25) & (var.DF.branch.trk.chi2pid.I2.chi2_muon < 10) &\
    (var.DF.branch.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.branch_mcs_range < 0.5)

hasstub = (var.DF.stub.l0_5cm.dqdx > 5e5) | (var.DF.stub.l1cm.dqdx > 3e5) |\
    (var.DF.stub.l2cm.dqdx > 2.5e5) | (var.DF.stub.l3cm.dqdx > 2e5)

stubid = ~hasstub

In [ ]:
othr_obj_cut = ~((var.DF.max_shw_len_primary > 10) | (var.DF.max_othr_trk_len > 20) |\
                 (var.DF.min_othr_chi2_proton < 80) | (var.DF.max_othr_chi2_muon > 40))

# othr_obj_cut = ~((var.DF.max_shw_len > 15) | (var.DF.max_othr_trk_len > 20) |\
#                  (var.DF.min_othr_chi2_proton_all < 80) | (var.DF.max_othr_chi2_muon > 40))

trunk_muid = (var.DF.trunk.trk.len > 25) & (var.DF.trunk.trk.chi2pid.I2.chi2_muon < 10) &\
    (var.DF.trunk.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.trunk_mcs_range < 0.5)
branch_muid = (var.DF.branch.trk.len > 25) & (var.DF.branch.trk.chi2pid.I2.chi2_muon < 10) &\
    (var.DF.branch.trk.chi2pid.I2.chi2_proton > 80) & (var.DF.branch_mcs_range < 0.5)

In [ ]:
nmu = mc.df.slc.truth.iscc & (np.abs(mc.df.slc.truth.pdg) == 14)
ns = (np.abs(mc.df.slc.truth.p0.pdg) == 3112) + (np.abs(mc.df.slc.truth.p0.pdg) == 3222) +\
      (np.abs(mc.df.slc.truth.p1.pdg) == 3112) + (np.abs(mc.df.slc.truth.p1.pdg) == 3222)
is_coh_like = (mc.df.slc.truth.max_proton_ke < 0.05) & (mc.df.slc.truth.npi + nmu + ns >= 2)

In [ ]:
when = cut.fiducial@var.slc &\
    cut.trkfiducial@var.DF.branch.trk & cut.trkfiducial@var.DF.trunk.trk &\
    (var.DF.openangle < 0.5*180/np.pi) &\
    trunk_muid & branch_muid & othr_obj_cut

In [ ]:
c = when(mc.df)

In [ ]:
(c & is_coh_like).sum() / c.sum()

In [ ]:
(2*onbeam.POT / mc.POT)*mc.df.wgt.cv[c].sum()

In [ ]:
2*onbeam.POT

In [ ]:
v = var.DF.beamangle(mc.df)
whens = [c & is_coh_like, c & ~is_coh_like]
labels = ["Coh-Like", "Not-Coh-Like"]

_ = plt.hist([v[w] for w in whens], label=labels, stacked=True, bins=np.linspace(0, 75, 11))
plt.legend()
plt.xlabel("$\\theta_\\mathrm{NuMI}$")
plt.ylabel("Events / Data POT")
plt.title("Selected Di-Muon Decay Candidates")
plt.axvline(7.5, color="red", linestyle=":", linewidth=2)
plt.text(0.01, 0.8, "Signal\nRegion", color="red", transform=plt.gca().transAxes, fontsize=12)

In [ ]:
for i, col in enumerate(cols):
    plt.figure(i)
    
    N,bins,_ = plt.hist(mc.df[c & ~is_coh_like][col], histtype="step")
    plt.hist(mc.df[c & is_coh_like][col], bins=bins, histtype="step")
    plt.xlabel(col)

In [ ]:
mc.df[(c & ~is_coh_like)].slc.truth.p0

In [ ]:
mc.df[(c & ~is_coh_like)].slc.truth.p1

In [ ]:
mc.df[(c & ~is_coh_like)].trunk.trk.truth

In [ ]:
mc.df[(c & ~is_coh_like)].branch.trk.truth

In [ ]:
mc.df[(c & ~is_coh_like) & (mc.df.branch.trk.truth.p.pdg == 2212)].branch.trk.chi2pid

In [ ]:
["%i:%i" % (r[1]["run"], r[1]["evt"]) for r in onbeam.hdrdf.loc[whichevt].iterrows()]

In [ ]:
FILELIST = "/icarus/app/users/gputnam/DP2023G/ICARUSRun2DataOnBeamMajority_stage0_disk_files_deduped.list"
NFILE_PER_JOB = 5

with open(FILELIST) as f:
    FILES = [l.rstrip("\n") for l in f]
FILES

In [ ]:
PROCS = [r[1]["proc"].values[0] for r in onbeam.hdrdf.loc[whichevt].iterrows()]
files_to_process = [FILES[P*NFILE_PER_JOB:(P+1)*NFILE_PER_JOB] for P in PROCS]
files_to_process = [f for files in files_to_process for f in files]
len(files_to_process)

In [ ]:
with open("/icarus/app/users/gputnam/DP2023G/INTERESTING_FILES.list", "w+") as f:
    for fname in files_to_process:
        f.write(fname + "\n")

In [ ]:
[r for r in onbeam.hdrdf.loc[whichevt].iterrows()][0][1]

In [ ]:
# Run 1 only test

In [ ]:
ifig = 0

onbeam_tmp = onbeam
offbeam_tmp = offbeam

onbeam = Dataset(onbeam.df[onbeam.df.Run1], onbeamR1.livetime, onbeamR1.POT)
offbeam = Dataset(offbeam.df[offbeam.df.Run1], offbeamR1.livetime, offbeamR1.POT)

for c, ctxt, cdir in zip(cuts[:3], cut_text[:3], cut_dir[:3]):
    ctxt = ctxt + "\nRun 1"
    cdir = cdir + "_Run1"
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for v, name, savename, bins in zip(plotvars, plotnames, plotsave, plotbins):
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices")
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("%s.svg" % savename)), bbox_inches="tight")
            plt.savefig(os.path.join(savedir, cdir, ("%s.pdf" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
onbeam = onbeam_tmp
offbeam = offbeam_tmp

In [ ]:
# Run 2 only test

In [ ]:
ifig = 0

onbeam = Dataset(onbeam.df[onbeam.df.Run2], onbeamR2.livetime, onbeamR2.POT)
offbeam = Dataset(offbeam.df[offbeam.df.Run2], offbeamR2.livetime, offbeamR2.POT)

for c, ctxt, cdir in zip(cuts[:3], cut_text[:3], cut_dir[:3]):
    ctxt = ctxt + "\nRun 2"
    cdir = cdir + "_Run2"
    print(ctxt)
    if dosave:
        os.makedirs(os.path.join(savedir, cdir), exist_ok=True)
    for v, name, savename, bins in zip(plotvars, plotnames, plotsave, plotbins):
        plt.figure(ifig)

        fig, (p0, p1) = mcdata_plot(v, bins, c, nucategories, name, "Slices")
        p0.text(0.9, 0.85, ctxt, transform=p0.transAxes, verticalalignment='top', horizontalalignment="right")
        
        if dosave: 
            plt.savefig(os.path.join(savedir, cdir, ("%s.pdf" % savename)), bbox_inches="tight")
        
        ifig += 1        
    if dosave: 
        plt.close("all")

In [ ]:
onbeam = onbeam_tmp
offbeam = offbeam_tmp